In [1]:
from meteostat import Point, Daily
from datetime import datetime
import numpy as np
import pandas as pd

start = datetime(2000, 1, 1)
end = datetime(2024, 5, 17)

# Raleigh, NC
raleigh = Point(35.7796, -78.6382)

data = Daily(raleigh, start, end)
data = data.fetch()


print(data)

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2000-01-01  10.0   0.6  19.4   0.0   0.0    NaN   4.7   NaN  1024.3   NaN
2000-01-02  13.9   6.1  21.1   0.0   0.0    NaN  12.2   NaN  1023.5   NaN
2000-01-03  18.3  12.8  23.3   0.0   0.0  208.0  16.6   NaN  1021.0   NaN
2000-01-04  16.7  10.6  22.8   8.6   0.0  218.0  25.6   NaN  1012.0   NaN
2000-01-05   3.9  -2.8  10.6   0.0   0.0    NaN   9.4   NaN  1024.3   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2024-05-13  18.8  10.6  26.7   0.0   0.0  133.0  10.4   NaN  1017.4   NaN
2024-05-14  29.8  22.2  37.2   0.0   NaN  105.0  11.9   NaN  1011.9   NaN
2024-05-15  28.4  22.8  35.0   0.0   NaN    0.0   4.7   NaN  1006.1   NaN
2024-05-16  27.4  22.2  33.9   0.0   NaN  351.0   5.2   NaN  1008.6   NaN
2024-05-17  27.9  21.7  36.7  19.1   NaN  152.0   6.9   NaN  1012.2   NaN

[8904 rows x 10 columns]


In [2]:
df = pd.DataFrame(data)

In [3]:
df.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2000-01-01,10.0,0.6,19.4,0.0,0.0,NaN,4.7,NaN,1024.3,NaN
2000-01-02,13.9,6.1,21.1,0.0,0.0,NaN,12.2,NaN,1023.5,NaN
2000-01-03,18.3,12.8,23.3,0.0,0.0,208.0,16.6,NaN,1021.0,NaN
2000-01-04,16.7,10.6,22.8,8.6,0.0,218.0,25.6,NaN,1012.0,NaN
2000-01-05,3.9,-2.8,10.6,0.0,0.0,NaN,9.4,NaN,1024.3,NaN


In [4]:
df.tail()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2024-05-13,18.8,10.6,26.7,0.0,0.0,133.0,10.4,NaN,1017.4,NaN
2024-05-14,29.8,22.2,37.2,0.0,NaN,105.0,11.9,NaN,1011.9,NaN
2024-05-15,28.4,22.8,35.0,0.0,NaN,0.0,4.7,NaN,1006.1,NaN
2024-05-16,27.4,22.2,33.9,0.0,NaN,351.0,5.2,NaN,1008.6,NaN
2024-05-17,27.9,21.7,36.7,19.1,NaN,152.0,6.9,NaN,1012.2,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8904 entries, 2000-01-01 to 2024-05-17
Freq: D
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tavg    8902 non-null   float64
 1   tmin    8904 non-null   float64
 2   tmax    8904 non-null   float64
 3   prcp    8904 non-null   float64
 4   snow    8900 non-null   float64
 5   wdir    4052 non-null   float64
 6   wspd    8903 non-null   float64
 7   wpgt    0 non-null      float64
 8   pres    8870 non-null   float64
 9   tsun    803 non-null    float64
dtypes: float64(10)
memory usage: 765.2 KB


In [6]:
print(df.isna().any().any())

True


In [7]:
df['rain'] = (df['prcp'] > 0).astype(int)

features = df[['prcp', 'wspd', 'pres']]
target = df['rain']

In [8]:
df.to_csv("Rainfall_OG.csv")

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [10]:
target.tail()

time
2024-05-13    0
2024-05-14    0
2024-05-15    0
2024-05-16    0
2024-05-17    1
Freq: D, Name: rain, dtype: int64

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LogisticRegression())

model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)